## Basic usage

In [1]:
import gnt
import pandas as pd

### Data
First, we load data from Najm et al. "Orthologous CRISPR–Cas9 enzymes for combinatorial genetic screens." These data are in the format guide 1, guide 2, gene 1, gene 2, followed by different conditions that were screene, which is generally the expected input for gnt.

In [2]:
lfcs = pd.read_csv('https://raw.githubusercontent.com/PeterDeWeirdt/bigpapi/master/data/processed/bigpapi_lfcs.csv')
lfcs

,U6 Sequence,H1 Sequence,U6 gene,H1 gene,Day 21_786O,Day 21_A375,Day 21_A549,Day 21_HT29,Day 21_Meljuso,Day 21_OVCAR8
0,AAAGTGGAACTCAGGACATG,AAAAAAAGAGTCGAATGTTTT,HPRT intron,6T,0.421135,0.250043,0.725424,0.635972,0.127104,0.245427
1,AAAGTGGAACTCAGGACATG,AAAGAGTCCACTCTGCACTTG,HPRT intron,UBC,0.040784,0.125369,0.343278,0.524569,-0.175984,0.371689
2,AAAGTGGAACTCAGGACATG,AACAGCTCCGTGTACTGAGGC,HPRT intron,CD81,0.711486,0.857567,1.513217,0.970841,0.630948,0.675330
3,AAAGTGGAACTCAGGACATG,AAGACGAAATTGAAGACGAAG,HPRT intron,CD81,0.451992,0.588394,1.283543,0.771713,0.409791,0.643640
4,AAAGTGGAACTCAGGACATG,AAGCGTACTGCTCATCATCGT,HPRT intron,HSP90AA1,0.477678,-0.652709,0.442170,0.021827,0.187209,-0.120412
...,...,...,...,...,...,...,...,...,...,...
9179,TTCTGACTACAACATCCAGA,TTGCTTTCATTTAATGCTACA,UBB,PARP2,-0.228266,-0.371034,-0.014272,0.570256,-0.437522,-0.717872
9180,TTCTGACTACAACATCCAGA,TTGGGACGAGTCCTGTGAGAA,UBB,IMPDH1,-0.178963,-0.024237,-0.323317,0.630812,-0.421810,0.197531
9181,TTCTGACTACAACATCCAGA,TTTAGGAATTGCTGTTGGGAC,UBB,HPRT intron,-0.266031,-0.429865,-0.145153,0.147415,-0.454209,-0.266580
9182,TTCTGACTACAACATCCAGA,TTTCCATCACTTGGTTGAATA,UBB,BCL2A1,-0.295739,-0.221819,-0.173578,0.231540,-0.430689,-0.377972


### Calculating residuals
From the log fold changes, we calculate **residuals** at the guide level. We reasoned that interactors for a given “anchor” guide would deviate from the expected range of LFCs of its “target” pairs. In order to generate an expectation for each anchor, we fit a linear model between the median LFC of targets paired with controls and the average LFC of constructs with both an anchor and target guide. Negative residuals from this line indicate a synthetic lethal relationship, whereas positive residuals represent a buffering interaction. We can then **scale** these residuals by the confidence interval of the linear model at each point.  

In [3]:
guide_residuals, model_info = gnt.get_guide_residuals(lfcs, ['CD81', 'HPRT intron'], scale=True)
guide_residuals.sort_values('scaled_residual_z')

,anchor_guide,target_guide,anchor_gene,target_gene,condition,lfc,lfc_target,residual,residual_z,mean_ci_lower,mean_ci_upper,ci,scaled_residual,scaled_residual_z
76126,GGGTAGTTCTCTCTTCATGGA,TCATCAACAGAGCCCGCCAA,WEE1,WEE1,Day 21_HT29,-5.676895,-1.874498,-2.627443,-6.821507,-3.182958,-2.915947,0.267011,-9.840207,-6.646213
3728,AAGCACCAGATCATGCACCG,TCGTGGGCACAAGGTCCTACA,MAP2K2,MAP2K1,Day 21_A549,-2.394606,-0.333970,-2.021550,-5.802628,-0.450153,-0.295959,0.154194,-13.110439,-6.561479
24468,CACAGGTCTTTCCTTATGGGA,ACACCACCTGAAGTGACTCG,CHEK1,CHEK1,Day 21_HT29,-3.539920,0.088388,-1.952853,-5.041612,-1.666680,-1.507453,0.159227,-12.264598,-6.373576
18051,ATCCAGACCATGATCACACA,CCAACGTGCTCCACCGAGATC,MAPK1,MAPK3,Day 21_A549,-2.178416,0.268715,-2.086836,-4.987346,-0.177208,-0.005952,0.171256,-12.185487,-6.278056
62395,GCAGTTTCTCATACCAGGTAG,CAGCATCGGATGCTTAGGAG,MTOR,MTOR,Day 21_HT29,-2.723119,-1.007286,-1.624228,-4.623783,-1.186494,-1.011288,0.175206,-9.270392,-6.077292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12605,AGCACAGTAGAACTAAGGGT,CCTTAGTTCTACTGTGCTCAT,BRCA2,BRCA2,Day 21_OVCAR8,0.715422,-0.206129,1.304595,5.799823,-0.639226,-0.539120,0.100106,13.032170,7.246571
57533,GATCACGCCTCCAGCCAGCTG,GGATCACGCCTCCAGCCAGC,CD81,CD81,Day 21_786O,3.943046,0.835260,2.536494,8.249168,1.313781,1.499322,0.185541,13.670784,7.821318
68454,GGATCACGCCTCCAGCCAGC,GATCACGCCTCCAGCCAGCTG,CD81,CD81,Day 21_786O,3.943046,1.040996,2.460461,8.347438,1.401317,1.563853,0.162536,15.137922,7.922174
58013,GATCACGCCTCCAGCCAGCTG,GGATCACGCCTCCAGCCAGC,CD81,CD81,Day 21_OVCAR8,4.011292,1.005960,2.393177,8.561875,1.523705,1.712525,0.188820,12.674394,8.055890


### Model info
We can also look at the fit of the linear model for each guide, by considering its R<sup>2</sup>. A low R<sup>2</sup> can represent a phenotypically dominant guide.

In [4]:
model_info

,R2,f_pvalue,anchor_guide,condition
0,0.950253,4.712468e-63,AAAAAAAGAGTCGAATGTTTT,Day 21_786O
1,0.973182,1.138096e-75,AAAAAAAGAGTCGAATGTTTT,Day 21_A375
2,0.891658,3.749836e-47,AAAAAAAGAGTCGAATGTTTT,Day 21_A549
3,0.975484,1.673119e-77,AAAAAAAGAGTCGAATGTTTT,Day 21_HT29
4,0.979379,4.906940e-81,AAAAAAAGAGTCGAATGTTTT,Day 21_Meljuso
...,...,...,...,...
1147,0.961787,1.932707e-68,TTTGCCTGTGAGGTGGCCGCC,Day 21_A375
1148,0.875427,2.675508e-44,TTTGCCTGTGAGGTGGCCGCC,Day 21_A549
1149,0.975138,3.233582e-77,TTTGCCTGTGAGGTGGCCGCC,Day 21_HT29
1150,0.961837,1.817770e-68,TTTGCCTGTGAGGTGGCCGCC,Day 21_Meljuso


### Combining scores at the gene level
We can then combine a statistic for a gene pair

$(\bar x - \mu)/(\sigma / \sqrt{n})$

Where $\bar x$, $\mu$, $\sigma$ are the sample mean, population mean, and population standard deviation of residuals, and $n$ is the number of guide pairs.

In [5]:
gene_scores = gnt.get_gene_residuals(guide_residuals, 'scaled_residual_z')
gene_scores.sort_values('z_score_scaled_residual_z')

,condition,gene_a,gene_b,lfc,guide_pairs,z_score_scaled_residual_z,z_score_scaled_residual_z_gene_a_anchor,z_score_scaled_residual_z_gene_b_anchor
1696,Day 21_HT29,MAPK1,MAPK3,-1.754885,18,-14.435802,-14.433085,-14.438520
2566,Day 21_OVCAR8,MAPK1,MAPK3,-0.616311,18,-13.451181,-13.087597,-13.814766
1261,Day 21_A549,MAPK1,MAPK3,-1.291028,18,-12.443262,-13.386711,-11.499813
2131,Day 21_Meljuso,MAPK1,MAPK3,-1.194016,18,-11.348421,-10.268737,-12.428104
610,Day 21_A375,BCL2L1,MCL1,-1.353692,18,-10.956703,-10.062025,-11.851381
...,...,...,...,...,...,...,...,...
594,Day 21_A375,BCL2L1,BCL2L1,-0.544783,9,6.099715,6.099715,6.099715
2582,Day 21_OVCAR8,MCL1,MCL1,0.318145,9,6.280362,6.280362,6.280362
1712,Day 21_HT29,MCL1,MCL1,0.396491,9,6.429010,6.429010,6.429010
2147,Day 21_Meljuso,MCL1,MCL1,-0.561327,9,6.920162,6.920162,6.920162
